# Minimum-Phase, Non-Minimum-Phase, and All-Pass Systems in the z-plane

This notebook continues the poles-and-zeros sequence and explains:

- What **minimum-phase** means (and why it matters)
- What **non-minimum-phase** means
- How zeros inside vs outside the unit circle affect **phase** but not **magnitude**
- What **all-pass filters** are
- Why zero reflection preserves magnitude but changes phase

This is the conceptual bridge between:
- Pole–zero geometry
- Phase response
- Group delay and time-domain behavior


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_zplane(poles=None, zeros=None, title="z-plane"):
    theta = np.linspace(0, 2*np.pi, 400)
    unit_x = np.cos(theta)
    unit_y = np.sin(theta)

    fig, ax = plt.subplots(figsize=(5,5))
    ax.axhline(0, linewidth=1)
    ax.axvline(0, linewidth=1)
    ax.plot(unit_x, unit_y)
    
    if zeros is not None and len(zeros) > 0:
        ax.plot(np.real(zeros), np.imag(zeros), 'o', label="zeros")
    if poles is not None and len(poles) > 0:
        ax.plot(np.real(poles), np.imag(poles), 'x', markersize=10, label="poles")

    ax.set_aspect('equal', 'box')
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.set_title(title)
    ax.set_xlabel("Re{z}")
    ax.set_ylabel("Im{z}")
    ax.legend()
    plt.show()


## 1) What does “minimum phase” mean?

Consider a **stable, causal** discrete-time LTI system.

It is called **minimum-phase** if:

- All **poles** are inside the unit circle (stability), and  
- All **zeros** are also inside the unit circle.

Equivalently:

> Among all systems with the same magnitude response \(|H(e^{j\omega})|\),  
> the minimum-phase system has the **smallest possible phase lag** and **smallest group delay**.

So “minimum phase” is about **time delay and transient behavior**, not just magnitude.


## 2) Why zero location affects phase but not magnitude (after reflection)

Take a zero at \(z_0\) with \(|z_0| \neq 1\).

Reflect it across the unit circle:

\[
z_0 \;\longleftrightarrow\; \frac{1}{z_0^*}
\]

Key result:

\[
|e^{j\omega} - z_0| = |e^{j\omega} - 1/z_0^*| \cdot |z_0|
\]

Up to a constant scale, the **magnitude response is unchanged**, but the **phase response changes**.

So:

- Zeros inside vs outside the unit circle  
- Same \(|H(e^{j\omega})|\)  
- Different phase and time-domain behavior


## 3) Example: same magnitude, different phase

We compare two FIR systems:

- System A: zero at \(z_0 = 0.5\) (inside unit circle) → minimum phase  
- System B: zero at \(1/z_0 = 2.0\) (outside unit circle) → non-minimum phase  

They will have:
- Same magnitude response (up to scale)  
- Different phase response  


In [ ]:
# Define two systems with reflected zeros
z0 = 0.5

# System A: zero inside unit circle
bA = np.array([1, -z0])

# System B: zero outside (reflected)
bB = np.array([1, -1/z0])

w = np.linspace(0, np.pi, 1024)
z = np.exp(1j*w)

HA = bA[0] + bA[1]*z**-1
HB = bB[0] + bB[1]*z**-1

# Magnitude
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(w, np.abs(HA), label="System A (min-phase)")
ax.plot(w, np.abs(HB), '--', label="System B (non-min-phase)")
ax.set_title("Magnitude responses (nearly identical shape)")
ax.set_xlabel("ω")
ax.set_ylabel("|H|")
ax.legend()
plt.show()

# Phase
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(w, np.unwrap(np.angle(HA)), label="System A (min-phase)")
ax.plot(w, np.unwrap(np.angle(HB)), '--', label="System B (non-min-phase)")
ax.set_title("Phase responses (very different)")
ax.set_xlabel("ω")
ax.set_ylabel("Phase (rad)")
ax.legend()
plt.show()

# Pole-zero plots
plot_zplane(poles=[], zeros=[z0], title="System A: zero inside (minimum phase)")
plot_zplane(poles=[], zeros=[1/z0], title="System B: zero outside (non-minimum phase)")


## 4) Time-domain consequence: group delay and pre-echo

Minimum-phase systems concentrate energy **as early as possible**.

Non-minimum-phase systems exhibit:

- Larger **group delay**
- Slower rise time
- Possible **pre-echo** (energy appearing before main lobe)

This is why minimum-phase designs are preferred in:

- Control systems  
- Inverse filtering  
- Seismology and deconvolution  
- Audio and imaging


## 5) All-pass filters

An **all-pass filter** has:

\[
|H(e^{j\omega})| = 1 \quad \text{for all } \omega
\]

It changes **only phase**, not magnitude.

### First-order all-pass

\[
H(z) = \frac{z^{-1} - a^*}{1 - a z^{-1}}, \quad |a|<1
\]

- Pole at \(z = a\)  
- Zero at \(z = 1/a^*\) (reciprocal location)  
- Mirror symmetry about unit circle


### Example: first-order all-pass filter

In [ ]:
# First-order all-pass
a = 0.7 * np.exp(1j*0.6)

# Frequency response
w = np.linspace(0, np.pi, 1024)
z = np.exp(1j*w)
H = (z**-1 - np.conj(a)) / (1 - a*z**-1)

# Magnitude
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(w, np.abs(H))
ax.set_title("All-pass magnitude response (|H| = 1)")
ax.set_xlabel("ω")
ax.set_ylabel("|H|")
plt.show()

# Phase
fig, ax = plt.subplots(figsize=(7,4))
ax.plot(w, np.unwrap(np.angle(H)))
ax.set_title("All-pass phase response")
ax.set_xlabel("ω")
ax.set_ylabel("Phase (rad)")
plt.show()

# Pole-zero plot
plot_zplane(poles=[a], zeros=[1/np.conj(a)], title="All-pass: pole-zero mirror pair")


## 6) Minimum-phase factorization

Any stable system can be factored as:

\[
H(z) = H_{\text{min}}(z) \cdot H_{\text{ap}}(z)
\]

where:

- \(H_{\text{min}}\): minimum-phase system (all zeros inside unit circle)  
- \(H_{\text{ap}}\): all-pass system (pure phase)

So:

- Magnitude response comes from \(H_{\text{min}}\)  
- Extra phase comes from \(H_{\text{ap}}\)

This is called **minimum-phase/all-pass decomposition**.


## 7) Big picture summary

| Concept | Meaning |
|--------|---------|
| Minimum-phase | All poles and zeros inside unit circle |
| Non-minimum-phase | At least one zero outside unit circle |
| Zero reflection | Preserves magnitude, changes phase |
| All-pass | Unit magnitude, arbitrary phase |
| Why it matters | Delay, transient shape, invertibility |

Key takeaways:

- Pole radius controls **stability and decay**  
- Zero location controls **phase and transient behavior**  
- Same magnitude response does **not** imply same time-domain response  
- Minimum-phase systems are the “fastest” possible realizations  
